In [47]:
from nba_api.stats.static import players
from nba_api.stats.endpoints import playergamelog
from nba_api.stats.library.parameters import SeasonAll
import pandas as pd 

In [48]:
player_dict = players.get_players()
kobe = [player for player in player_dict if player['full_name'] == 'Kobe Bryant'][0]
kobe_id = kobe['id']

In [49]:
gamelog_kobe = playergamelog.PlayerGameLog(player_id=kobe_id, season = SeasonAll.all)

In [50]:
df_kobe_games_all = gamelog_kobe.get_data_frames()[0]

In [62]:
df_kobe_games_all['OPPPONENTS'] = df_kobe_games_all['MATCHUP'].apply(lambda x: x.split()[-1])

In [76]:
# clean up opponent abbreviation
remap = {"UTH": "UTA", "SAN": "SAS", "PHO": "PHX",  "NJN": "BKN", "NOK": "NOP", "NOH":"NOP", "SEA": "OKC", "CHH": "CHA", "VAN": "MEM"}
df_kobe_games_all['OPPPONENTS'].replace(remap, inplace=True)
df_kobe_games_all['OPPPONENTS'].value_counts()

LAC    69
GSW    67
MEM    67
DEN    66
OKC    65
SAC    64
MIN    64
PHX    64
POR    62
SAS    61
HOU    61
DAL    60
UTA    60
NOP    41
IND    34
CLE    34
NYK    34
MIA    33
TOR    33
DET    33
MIL    32
PHI    32
WAS    32
CHI    31
CHA    31
BOS    31
ORL    29
ATL    29
BKN    27
Name: OPPPONENTS, dtype: int64

In [77]:
df_kobe_games_all.to_csv("game_stats.csv")